# TF  jensen-shannon diverse


In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc


In [3]:
Adata_File = './Frog.h5ad'
Meta_File = './Frog_anno.txt' # contain four columns: Cell , Species, Celltype and Cellcluster.
Meta_File2 = './Frog_annotation_bionary.txt' # contain binary result with row cell lineage and column cellname.
TF_File    = './Frog_TF.txt'
OUT_File1  = 'TF_JSD.out'
OUT_File2  = 'TF_JSDR.out'

In [ ]:
adata_SC = sc.read_h5ad(Adata_File)
adata_SC_df = adata_SC.to_df().T
adata_SC_df.columns = adata_SC_df.columns.astype("str")
adata_SC_df.index = adata_SC_df.index.astype("str")
adata_SC_df.shape

In [ ]:
meta_SC = pd.read_table(Meta_File,sep="\t",header=0,index_col="Cell")
cell_index1 = adata_SC_df.columns.intersection(meta_SC.index)
cell_index1.shape[0]

In [ ]:
TF_S = pd.read_table(TF_File,sep="\t",header=0)
TF_S.shape

In [ ]:
TF_S[:2]

In [ ]:
gene_index = adata_SC_df.index.intersection(TF_S['Symbol']) ##CHECK
gene_index.shape

In [12]:
adata_SC_df_use = adata_SC_df.loc[gene_index,:]

In [13]:
meta_SC2 = pd.read_table(Meta_File2,sep="\t")
#meta_SC2 = meta_SC2.loc[:,cell_index]

In [14]:
Input1 = adata_SC_df_use # p matrix
Input2 = meta_SC2 # q matrix

In [15]:
Input1 = Input1.div(Input1.sum(axis=1),axis='rows')
Input2 = Input2.div(Input2.sum(axis=1),axis='rows')

In [ ]:
Input1.shape


In [ ]:
Input2.shape

In [18]:
JSD_list = []
JSDR_list = []

In [19]:
import math
for i in range(Input1.shape[0]):
    for j in range(Input2.shape[0]):
        p = np.asarray(Input1.iloc[i,:]) + np.spacing(1)
        q = np.asarray(Input2.iloc[j,:]) + np.spacing(1)
        M=(p+q)/2 + np.spacing(1)   
        jsd_value=0.5*np.sum(p*np.log2(p/M))+0.5*np.sum(q*np.log2(q/M))
        jsd_r = 1- math.sqrt(jsd_value)
        JSD_list.append([Input1.index[i],Input2.index[j],jsd_value])
        JSDR_list.append([Input1.index[i],Input2.index[j],jsd_r])

In [20]:
JSD_pd = pd.DataFrame(JSD_list,columns=['Cell','Celltype','JSD'])
JSDR_pd = pd.DataFrame(JSDR_list,columns=['Cell','Celltype','JSDR'])

In [21]:
JSD_pd.to_csv(OUT_File1,index = 0,sep="\t")
JSDR_pd.to_csv(OUT_File2,index = 0,sep="\t")

In [22]:
#end